<a href="https://colab.research.google.com/github/LeticiaAmim/Info3DBlet/blob/main/TrabalhoBD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
connection = sqlite3.connect("financas.sqlite")

query = """
create table usuario(
	id_usuario integer primary key autoincrement,
  nome varchar(100),
  email varchar(100),
  celular char(15),
  senha varchar(100)
);
"""
execute_query(connection, query)

query = """
create table card(
	id_card integer primary key autoincrement,
  descricao text,
  resumo varchar (140),
  titulo varchar(100)
);
"""
execute_query(connection, query)

query = """
create table avaliacao(
	id_avaliacao integer primary key autoincrement,
  nota float,
  comentario varchar(140),
  dataAvaliacao date
);
"""
execute_query(connection, query)

query = """
create table vocabulario(
	id_vocabulario integer primary key autoincrement,
  tituloVocabulario varchar(140)
);
"""
execute_query(connection, query)

# -- Relacionamento
query = """
create table avaliar (
    id_usuario integer,
    id_avaliacao integer,
    primary key (id_usuario, id_avaliacao),
    foreign key (id_usuario) references Usuario(id_usuario),
    foreign key (id_avaliacao) references Avaliacao(id_avaliacao)
);
"""
execute_query(connection, query)

query = """
create table visualizar (
    id_usuario integer,
    id_vocabulario integer,
    primary key (id_usuario, id_vocabulario),
    foreign key (id_usuario) references Usuario(id_usuario),
    foreign key (id_vocabulario) references Vocabulario(id_vocabulario)
);
"""
execute_query(connection, query)

query = """
create table favoritar (
    id_usuario integer,
    id_card integer,
    primary key (id_usuario, id_card),
    foreign key (id_usuario) references Usuario(id_usuario),
    foreign key (id_card) references Card(id_card)
);
"""
execute_query(connection, query)

Atividade 01

Consulta 01

Objetivo da consulta (contexto de PI3):
Contar quantos cards um usuário adicionou aos favoritos.

Descrição da ação esperada ao rodar a consulta:
A consulta usa COUNT() para contar os registros da tabela favoritar por usuário, retornando quantos cards aquele usuário favoritou.


In [ ]:
SELECT id_usuario, COUNT(id_card) AS total_favoritos
FROM favoritar
GROUP BY id_usuario;

CONSULTA 2

Objetivo da consulta (contexto de PI3):
Calcular a média das avaliações feitas pelos usuários para fins de análise de satisfação geral da aplicação.

Descrição da ação esperada ao rodar a consulta:
A consulta usa AVG() para calcular a média de notas da tabela avaliacao.

In [ ]:
SELECT AVG(nota) AS media_avaliacoes
FROM avaliacao;


Atividade 02
CONSULTA COM 3 ENTIDADES

Objetivo da consulta (contexto de PI3):
Listar os usuários que avaliaram cards, mostrando o nome do usuário, a nota dada e a data da avaliação.

Descrição da ação esperada ao rodar a consulta:
Relacionar as tabelas usuario, avaliar e avaliacao para mostrar quem avaliou o quê, com qual nota e em qual data.

In [ ]:
SELECT u.nome, a.nota, a.dataAvaliacao
FROM usuario u
JOIN avaliar av ON u.id_usuario = av.id_usuario
JOIN avaliacao a ON av.id_avaliacao = a.id_avaliacao;

Atividade 03

Justificativa para que a consulta seja uma View (Contexto de PI3):

Consultar as avaliações feitas por usuários é algo frequente para exibir em painéis administrativos ou em perfis de usuário. Por isso, é útil criar uma VIEW para facilitar o acesso.

Descrição da ação esperada ao rodar a consulta:

Retornar nome do usuário, nota, comentário e data da avaliação.

In [ ]:
VIEW
CREATE VIEW vw_avaliacoes_usuarios AS
SELECT u.nome, a.nota, a.comentario, a.dataAvaliacao
FROM usuario u
JOIN avaliar av ON u.id_usuario = av.id_usuario
JOIN avaliacao a ON av.id_avaliacao = a.id_avaliacao;

SELECT PARA EXIBIR TODOS OS DADOS DA VIEW
SELECT * FROM vw_avaliacoes_usuarios;

Atividade 04

Justificativa para que a operação seja um Procedure (Contexto de PI3):

Cadastrar uma nova avaliação é uma ação comum na aplicação, e o procedimento automatiza isso para evitar repetição de código e garantir que todos os dados necessários sejam inseridos corretamente.

Descrição da ação esperada ao executar o procedure:

Inserir uma nova avaliação e vincular essa avaliação a um usuário.

In [ ]:
CREATE PROCEDURE sp_inserir_avaliacao (
    IN p_nota FLOAT,
    IN p_comentario VARCHAR(140),
    IN p_data DATE,
    IN p_id_usuario INTEGER
)
BEGIN
    INSERT INTO avaliacao (nota, comentario, dataAvaliacao)
    VALUES (p_nota, p_comentario, p_data);

    INSERT INTO avaliar (id_usuario, id_avaliacao)
    VALUES (p_id_usuario, last_insert_rowid());
END;

EXECUÇÃO DO PROCEDURE (representação)
CALL sp_inserir_avaliacao(4.5, 'Muito bom!', '2025-09-15', 1);

Atividade 05

Justificativa para que a operação seja uma Function (Contexto de PI3):

Calcular quantos cards um usuário favoritou é uma lógica que pode ser usada em perfis, dashboards e relatórios. Criar uma função facilita o reuso.

Descrição da operação realizada e valor retornado pela Function:

A função recebe um id_usuario e retorna a quantidade de cards que ele favoritou.

In [ ]:
FUNCTION (em SQL padrão)
-- Padrão SQL, não suportado diretamente no SQLite
CREATE FUNCTION fn_total_favoritos (p_id_usuario INT)
RETURNS INT
BEGIN
    DECLARE total INT;
    SELECT COUNT(*) INTO total
    FROM favoritar
    WHERE id_usuario = p_id_usuario;
    RETURN total;
END;

USO DA FUNCTION (representação)
SELECT nome, fn_total_favoritos(id_usuario) AS qtd_favoritos
FROM usuario;

Atividade 06

Justificativa para que a operação seja um Trigger (Contexto de PI3):

Manter um histórico de ações dos usuários, como favoritarem cards, é essencial para auditoria e análise de comportamento.

Descrição do evento que dispara o Trigger e das operações que ele executa:

Quando um card é favoritado, o trigger registra essa ação na tabela log_favoritos.


In [ ]:
TRIGGER
CREATE TABLE log_favoritos (
    id_usuario INTEGER,
    id_card INTEGER,
    data_favorito DATETIME
);

CREATE TRIGGER trg_log_favorito
AFTER INSERT ON favoritar
FOR EACH ROW
BEGIN
    INSERT INTO log_favoritos (id_usuario, id_card, data_favorito)
    VALUES (NEW.id_usuario, NEW.id_card, datetime('now'));
END;

COMANDO QUE DISPARA O TRIGGER
INSERT INTO favoritar (id_usuario, id_card)
VALUES (1, 2);